# Импорт библиотек

In [1]:
import pandas as pd
from tqdm.notebook import trange, tqdm_notebook
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import random
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

In [2]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 0.0005

SEED = 69
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

data_dir = './data/classed_type_train/'
test_dir = './data/test2/'

cuda


## Модель `cnn_v2`

In [3]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(192), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(45),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(192),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(192),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [4]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    val_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    trainloader = DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    valloader = DataLoader(val_data,
                   sampler=val_sampler, batch_size=batch_size)
    return trainloader, valloader

In [5]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))
train_loader.dataset.class_to_idx

['ER', 'NR']
['ER', 'NR']
166
42


{'ER': 0, 'NR': 1}

In [6]:
class ConvNet(nn.Module): 
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop1 = nn.Dropout2d(p=0.25)
            self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop2 = nn.Dropout2d(p=0.25)
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop3 = nn.Dropout2d(p=0.25)
            self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.fc1 = nn.Sequential(
                nn.Linear(12800, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
            )
            self.fc2 = nn.Sequential(
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
            )
            self.out = nn.Linear(1024, 2)
        
        def forward(self, x):
            x = self.drop1(self.conv1(x))
            x = self.drop2(self.conv2(x))
            x = self.drop3(self.conv3(x))
            x = self.conv4(x)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)

            return F.log_softmax(x, dim=1)

In [7]:
net = ConvNet()
net = net.to(device)

#Задаю функцию потерь
criterion = nn.NLLLoss()

#Задаю оптимизатор
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

for epoch in trange(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(train_loader)):
        # Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net(images)
        loss = criterion(outputs, labels)

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total


    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [1/20], Loss: 0.6046, Accuracy: 64.71%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [2/20], Loss: 0.1069, Accuracy: 95.83%


...

Epoch [18/20], Loss: 0.0139, Accuracy: 99.54%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [19/20], Loss: 0.0126, Accuracy: 99.58%


  0%|          | 0/166 [00:00<?, ?it/s]

Epoch [20/20], Loss: 0.0099, Accuracy: 99.70%


In [8]:
# 99.48%
# 99.78%

In [9]:
#Проверка на валидационной выборке
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm_notebook(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

  0%|          | 0/42 [00:00<?, ?it/s]

Test Accuracy of the model on the test images: 100.0 %


## Submission

In [11]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [12]:
%%time
net.eval()
with torch.no_grad():
    for i, (img, name) in enumerate(iter(test_loader)):
        img = img.to(device)
        outputs = net(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = list(map(int, predicted.cpu()))
        dict_pred["classification_predictions"].extend(predicted)
print(len(dict_pred['id']))
print(len(dict_pred['classification_predictions']))

16564
16564
Wall time: 2min 18s


In [13]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "classification_predictions"])
data_frame.to_csv('./submissions/cnn-v2-type.csv', index=False, header=True)